In [ ]:
!pip install accelerate transformers einops langchain xformers bitsandbytes sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install gradio faiss-gpu  PyMuPDF gdown pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 39.6 MB/s eta 0:

## 1st

In [ ]:
import gradio as gr
import torch
from transformers import pipeline, AutoTokenizer,AutoConfig,AutoModelForCausalLM, BitsAndBytesConfig, StoppingCriteria, StoppingCriteriaList
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# Initialize the Hugging Face Pipeline
model_id = 'NousResearch/Llama-2-7b-chat-hf'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_config = AutoConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
# Define custom stopping criteria class
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

In [ ]:
stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
!gdown --id 1Tmo94S0ydWaI4HP-TEAFlgLhfJUI3Dok -O documents.zip
!unzip -q documents.zip -d documents

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Tmo94S0ydWaI4HP-TEAFlgLhfJUI3Dok
To: /content/documents.zip
100% 4.68M/4.68M [00:00<00:00, 148MB/s]


In [ ]:
generate_text = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.1,
    max_new_tokens=512,
    repetition_penalty=1.1
)

# Initialize LangChain components
llm = HuggingFacePipeline(pipeline=generate_text)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = DirectoryLoader('./documents/documents/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
all_splits = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
vectorstore = FAISS.from_documents(all_splits, embeddings)

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)
chat_history = []

# Define the Gradio interface function
def answer_question(query):
    result = chain({"question": query, "chat_history": chat_history})
    # chat_history.append((query, result["answer"]))
    return result['answer']

In [ ]:
import time
import gradio as gr

def slow_echo(question, history):
    for i in range(len(question)):
        # time.sleep(0.05)
        ans = answer_question(question)
        yield "Answer: " + ans

title = "CoalWizard 🧙🏻‍♂️"
description = "Welcome to the CoalWizard! Ask me anything about Coal !"

demo = gr.ChatInterface(
    slow_echo,
    title=title,
    description=description,
    examples=[["What is the definition of an *open cast working* according to the Mines Act?",
               "What is the significance of setting the time references in the Mines Act to Indian standard time?",
               "What is the role of a Chief Inspector of Mines in the context of the Mines Act?",
               "How is the Chairman of a Committee under the Mines Act appointed?",
               "What is the composition of the Committee constituted under Section 12 of the Mines Act?",
               "What powers do Committees constituted under Section 12 of the Mines Act have?"
]],
theme="default").queue()

if __name__ == "__main__":
    demo.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7d6e347edd5890f12c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7d6e347edd5890f12c.gradio.live


In [ ]:
import time
import gradio as gr

def slow_echo(question, history):
    for i in range(len(question)):
        # time.sleep(0.05)
        ans = answer_question(question)
        yield "Answer: " + ans

title = "Mining Regulatory Chatbot 🏭"
# description = "Welcome to the Mining Regulatory Chatbot! Ask me anything about Acts, Rules, and Regulations applicable to the Mining Industry."

article = """
## Meet Your Mining Industry Expert

Meet our cutting-edge Chatbot powered by Lanchain and Llama 2 – Your 24/7 mining industry expert, providing instant answers to all your queries on Acts, Rules, Regulations, and more!

The mining industry is subject to a complex web of Acts, Rules, and Regulations that govern various aspects of its operations. These legal frameworks are crucial for ensuring safety, environmental protection, and fair practices within the industry.

### Key Topics Covered by Mining Regulations:

1. **Safety Regulations:** Mining operations involve inherent risks. Regulations mandate safety measures to protect workers and prevent accidents.

2. **Environmental Compliance:** Mining activities can have a significant impact on the environment. Regulations require companies to mitigate their environmental footprint.

3. **Licensing and Permitting:** Companies must obtain the necessary licenses and permits to operate legally.

4. **Labor Laws:** Regulations cover labor rights, wages, and working conditions for mining sector employees.

5. **Community Engagement:** Many regulations emphasize community engagement and corporate social responsibility.

Explore the Mining Regulatory Chatbot to get answers to your specific questions regarding these regulations.

"""
examples = [
    "What is the definition of an *open cast working* according to the Mines Act?",
    "What is the significance of setting the time references in the Mines Act to Indian standard time?",
    "What is the role of a Chief Inspector of Mines in the context of the Mines Act?",
    "How is the Chairman of a Committee under the Mines Act appointed?",
    "What is the composition of the Committee constituted under Section 12 of the Mines Act?",
    "What powers do Committees constituted under Section 12 of the Mines Act have?"
]

demo = gr.ChatInterface(
    slow_echo,
    title=title,
    # article=article,
    description=article,
    examples=examples,
    theme="default").queue()

if __name__ == "__main__":
    demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://54a4a9c538433e6404.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Final

In [ ]:
import gradio as gr
import os
import torch
from transformers import pipeline, AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig, StoppingCriteria, StoppingCriteriaList
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain

# Initialize the Hugging Face Pipeline
model_id = 'NousResearch/Llama-2-7b-chat-hf'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_config = AutoConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define custom stopping criteria class
stop_list = ['\nHuman:', '\n```\n']
stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

# Initialize text generation pipeline
generate_text = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.1,
    max_new_tokens=512,
    repetition_penalty=1.1
)


!gdown --id 1Tmo94S0ydWaI4HP-TEAFlgLhfJUI3Dok -O documents.zip
!unzip -q documents.zip -d documents

# Initialize LangChain components for retrieval
llm = HuggingFacePipeline(pipeline=generate_text)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = DirectoryLoader('./documents/documents/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
all_splits = text_splitter.split_documents(documents)

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
vectorstore = FAISS.from_documents(all_splits, embeddings)

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

# Define the Gradio interface function
def answer_question(query, uploaded_file):
    # Perform text generation based on the query
    generation_result = chain({"question": query})

    # Extract the answer from the generation result
    answer = generation_result['answer']

    # Process the uploaded PDF file here and extract information or perform further analysis

    # Return the answer to the user
    return answer

# Gradio interface for chatbot
def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.update(value="", interactive=False)

def add_file(history, file):
    history = history + [((file.name,), None)]
    return history

def bot(history):
    query = history[-1][0]
    uploaded_file = history[-2][0][0]
    response = answer_question(query, uploaded_file)
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="CoalWizard 🧙🏻‍♂️",
        bubble_full_width=False,
    )

    with gr.Row():
        txt = gr.Textbox(
            scale=4,
            show_label=False,
            placeholder="Enter text and press enter, or upload a PDF",
            container=False,
        )
        btn = gr.UploadButton("📁", file_types=["pdf"])  # Allow only PDF files

    txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(
        bot, chatbot, chatbot
    )
    txt_msg.then(lambda: gr.update(interactive=True), None, [txt], queue=False)
    file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

demo.queue()
if __name__ == "__main__":
    demo.launch()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Tmo94S0ydWaI4HP-TEAFlgLhfJUI3Dok
To: /content/documents.zip
100% 4.68M/4.68M [00:00<00:00, 191MB/s]


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c5300df105c05201be.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
